In [ ]:
# sum all file
# set dtype
# date process
# delete threshold=0.2(0.5 ดีไหม) n_null
# merge column
# llm about date with column relation -> interpolate but no impute

# null เป็นประโยชน์ เย่ yeah

# personal information

# sum -> score -> corr
# reduce mem -> cat ตาม sub-group

# load data

In [ ]:
!wget https://storage.googleapis.com/ss4-exp-datasource/datascience/home-credit-credit-risk-model-stability.zip

--2024-05-21 16:39:54--  https://storage.googleapis.com/ss4-exp-datasource/datascience/home-credit-credit-risk-model-stability.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3375704785 (3.1G) [application/x-zip-compressed]
Saving to: ‘home-credit-credit-risk-model-stability.zip’

home-credit-credit- 100%[===================>]   3.14G   231MB/s    in 15s     

2024-05-21 16:40:09 (215 MB/s) - ‘home-credit-credit-risk-model-stability.zip’ saved [3375704785/3375704785]



In [ ]:
!unzip /content/home-credit-credit-risk-model-stability.zip

# grouping

In [ ]:
import pandas as pd
import polars as pl
import numpy as np

In [ ]:
def dtype_to_right(df):
    for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.Utf8))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))
    return df

In [ ]:
def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data, cat_cols

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype
        #print(str(col_type))
        if str(col_type)=="category" or str(col_type)=="datetime64[ms]":
            continue

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2
    return df

In [ ]:
def read_allfiles(path_ls):
    chunks = []

    for p in path_ls:
        df = pl.read_parquet(p)
        df = df.pipe(dtype_to_right)
        chunks.append(df)

    final_df = pl.concat(chunks, how="vertical_relaxed")
    final_df = final_df.unique(subset=["case_id"])

    final_df, cols = to_pandas(final_df)
    final_df = reduce_mem_usage(final_df)

    return final_df

In [ ]:
import glob
all_parquet_path = glob.glob("/content/parquet_files/train/*")
all_parquet_path.sort()
#all_parquet_path

In [ ]:
all_parquet_path[-1]

'/content/parquet_files/train/train_tax_registry_c_1.parquet'

In [ ]:
all_path = all_parquet_path[-1]

df = read_allfiles([all_path])
df.shape

(482265, 5)

In [ ]:
# count null
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing}, index=df.columns)
missing_value_df

,percent_missing
case_id,0.0
employername_160M,0.0
num_group1,0.0
pmtamount_36A,0.0
processingdate_168D,0.0


In [ ]:
list(df.columns)